In [3]:
# Initialize Otter
import otter
grader = otter.Notebook("ps7.ipynb")

### Back to NYC flights dataset!
We will again look into the NYC flights dataset using Pandas!

We will run many of the same queries we ran using SQL earlier but now using Pandas API
to get the same result

### Question 1
Load the dataset from flights.csv file given with this file into a dataframe variable named `df`


In [4]:
import pandas as pd
df = pd.read_csv('flights.csv')

In [5]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2
Get the total count of flights for each month and display the top three months with the highest count.

Hint: We looked into the groupby function in class. I also encourage you to look into `value_counts` method that can applied to both Series and DataFrames: https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html#pandas.Series.value_counts

In [32]:
print(df)
#df.describe

        year  month  day  dep_time  ...  distance  hour  minute     time_hour
0       2013      1    1     517.0  ...    1400.0   5.0    15.0  1.357034e+09
1       2013      1    1     533.0  ...    1416.0   5.0    29.0  1.357034e+09
2       2013      1    1     542.0  ...    1089.0   5.0    40.0  1.357034e+09
3       2013      1    1     544.0  ...    1576.0   5.0    45.0  1.357034e+09
4       2013      1    1     554.0  ...     762.0   6.0     0.0  1.357038e+09
...      ...    ...  ...       ...  ...       ...   ...     ...           ...
336771  2013      9   30       NaN  ...     213.0  14.0    55.0  1.380564e+09
336772  2013      9   30       NaN  ...     198.0  22.0     0.0  1.380593e+09
336773  2013      9   30       NaN  ...     764.0  12.0    10.0  1.380557e+09
336774  2013      9   30       NaN  ...     419.0  11.0    59.0  1.380553e+09
336775  2013      9   30       NaN  ...     431.0   8.0    40.0  1.380542e+09

[336776 rows x 19 columns]


In [47]:
monthly_count = df.groupby('month').size().sort_values(ascending=False).head(3)
monthly_count

month
7     29425
8     29327
10    28889
dtype: int64

In [46]:
grader.check("q2")

q2 results: All test cases passed!

### Question 3
First get a new dataframe with the three columns of interest 'carrier', 'month' and 'dep_delay'. Then compute the average dep_delay for each carrier and for each month. Rank the departure delays/month and filter the 1st rank carrier for each month (carrier with the lowest dep_delay for the month will get the 1st rank). Show the results in ascending order of the month

Hint: Some of the functions that can help are rank(), unstack(), stack(). 

In [147]:
selected_columns = df[['carrier', 'month', 'dep_delay']]
grouped = selected_columns.groupby(['carrier', 'month']).agg(avg_dep_delay=('dep_delay', 'mean')).reset_index()
grouped['rank'] = grouped.groupby('month')['avg_dep_delay'].rank(method='min')
filtered = grouped[grouped['rank'] == 1]
sorted_df = filtered.sort_values(by='month')
avg_delay = sorted_df.set_index(['carrier', 'month'])

In [148]:
grader.check("q3")

q3 results: All test cases passed!

### Question 4
Find the 5 top destinations in this dataset (most number of flights to). Filter only those flights to these top 5 destinations and then show the distribution of carriers on this filtered data

In [178]:
#df
import numpy as np
top_dest = df['dest'].value_counts().nlargest(5).index
top_dest_carriers = df[df['dest'].isin(top_dest)]
top_carrier_count = top_dest_carriers['carrier'].value_counts()
#top_carrier_count.index


In [179]:
grader.check("q4")

q4 results: All test cases passed!

### Question 5

Get the total count of cancelled flights in this dataset. We'll say that a cancelled flight has dep_time missing.


In [184]:
cancelled_flight_count = df['dep_time'].isna().sum()


In [185]:
grader.check("q5")

q5 results: All test cases passed!

### Question 6
Count all the destinations that have a letter 'K' anywhere in the name code.

Hint: Look into the `str` attribute of the Series to apply string methods: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html

In [193]:
#df.columns
dest_with_k = df['dest'].str.contains('K').sum()
dest_with_k



4589

In [194]:
grader.check("q6")

q6 results: All test cases passed!

### Question 7
Create a new column called speed that computes the speed as distance/air_time

In [195]:
df['speed'] = df['distance'] / df['air_time']

In [196]:
grader.check("q7")

q7 results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 8
Segment the dep_time 24 hours into 
* night from 12 midnight to 5 am
* morning from 5:01 am to 12 noon
* afternoon from 12.01 noon to 5 pm
* evening from 5.01 pm to 12 am

And then run a query to find the total flights in each segment and save that value in the given variable.

Hint: Look into the `cut` function: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html



In [230]:
counts = (df.assign(segment=pd.cut(df['dep_time'], 
                            bins=[0, 501, 1201, 1700, 2400], 
                            labels=['night', 'morning', 'afternoon', 'evening'],
                            right=False)).groupby('segment').size())
parts_of_day_count = counts.to_dict()
print(parts_of_day_count)

{'night': 1487, 'morning': 129939, 'afternoon': 98214, 'evening': 98852}


In [221]:
grader.check("q8")

q8 results:
    q8 - 1 result:
        ❌ Test case failed
        Trying:
            assert parts_of_day_count['morning'] == 129939
        Expecting nothing
        ok
        Trying:
            assert parts_of_day_count['afternoon'] == 98733
        Expecting nothing
        ok
        Trying:
            assert parts_of_day_count['evening'] == 98362
        Expecting nothing
        **********************************************************************
        Line 3, in q8 0
        Failed example:
            assert parts_of_day_count['evening'] == 98362
        Exception raised:
            Traceback (most recent call last):
              File "/opt/conda/lib/python3.11/doctest.py", line 1351, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q8 0[2]>", line 1, in <module>
                assert parts_of_day_count['evening'] == 98362
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
            AssertionError
        Trying:
            assert parts_of_day_count['night'] == 1487
        Expecting nothing
        ok

<!-- END QUESTION -->



## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)